In [1]:
!pip install pyDecision

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)


In [2]:
# Required Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pyDecision.algorithm import electre_tri_b

In [33]:
# Load subset
df = pd.read_csv('/project/processed_data/grouped_bymean_data.csv')
df.head(3)

,geometry,Storefront,Visible Sign of Business,Business WiFi,Business tables,Business Plugs,Business Internal CCTV,External CCTV,Number of female employees,Accepts credit card,...,Injuries,Collision,Run over,Metro,Metrobus,Trolleybus,RTP,Concessioned,e-bike,Total coverage of transport
0,POLYGON ((483895.6627641306 2147977.5123609374...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.5,1.0,1.0,0.0,1.0,1.0,0.5,4.5
1,"POLYGON ((483983.9277624827 2148086.839068036,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.5,4.5
2,"POLYGON ((484015.5864170455 2148025.338677514,...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,5.0


Drop geometry

In [34]:
df= df.drop(['geometry','Flooding'], axis=1)

In [35]:
conditions = [
    (df['excellent street lighting'] == 1),
    (df['good street lighting'] == 1),
    (df['regular street lighting'] == 1)
]

choices = [3, 2, 1]

df['lighting'] = np.select(conditions, choices, default=0)


In [39]:
df= df.drop(['excellent street lighting','good street lighting','regular street lighting'], 
            axis=1)

In [41]:
conditions = [
    (df['fully painted crosswalk'] == 1),
    (df['partially painted crosswalk'] == 1),
    (df['not painted crosswalk'] == 1)
]

choices = [3, 2, 1]

df['crosswalk'] = np.select(conditions, choices, default=0)

In [42]:
df= df.drop(['fully painted crosswalk', 'not painted crosswalk',
       'partially painted crosswalk'], 
            axis=1)

In [43]:
df.columns

Index(['Storefront', 'Visible Sign of Business', 'Business WiFi',
       'Business tables', 'Business Plugs', 'Business Internal CCTV',
       'External CCTV', 'Number of female employees', 'Accepts credit card',
       'Incline', 'Width', 'Ramp conditions', 'Ramp // Crosswalk',
       'Ramp obstacles', 'Crosswalk width', 'Vertical signs',
       'Crossing length', 'Access to people with disabilities',
       'Broken public property', 'Incivility', 'Obstacles',
       'Physical aspects in poor condition', 'Poor infrastructure',
       'Positive elements', 'Broken drain-cover', 'Drug addicts',
       'Federal Judiciary', 'Panic button', 'Vandalism', 'graffiti',
       'homelessness', 'inclined ramp', 'indication of security',
       'mobile vendors', 'police presence', 'risky building', 'risky pavement',
       'risky road', 'tourist map', 'trash', 'vehicle abandoned',
       'Coffee Shop', 'Food joint', 'Ice Cream Shop', 'Lighting Stores',
       'Restaurant', 'Open-Weekdays', 'Open-We

# Normalization

In [44]:
# Set numerical columns
df_columns_to_normalize = ['Number of female employees', 'Incline','Width','Ramp conditions',
                           'Crosswalk width','Public WiFi', 'number of bikes', 
                           'Employment','Broken public property','lighting', 'crosswalk',
                           'Crime Degree', 'school_count', 'hospitals_count',
                           'Injuries', 'Collision', 'Run over', 'Total coverage of transport']

n_scaler = MinMaxScaler()

# Create a copy of DataFrame
df_normalized = df.copy()

df_normalized[df_columns_to_normalize] = n_scaler.fit_transform(df_normalized[df_columns_to_normalize])

In [46]:
# Reverse the scale of the minimization criteria by subtracting from 1
minimization_criteria = ['Crime Degree', 'Broken public property', 'homelessness',
                         'risky road','Physical aspects in poor condition', 
                         'Incivility', 'Obstacles','risky building', 'risky pavement',
                        'police presence','Ramp obstacles','Poor infrastructure',
                        'Drug addicts','trash', 'vehicle abandoned',
                        'Deaths', 'Injuries', 'Collision','Run over',]

df_normalized[minimization_criteria] = 1 - df_normalized[minimization_criteria]

In [49]:
column_names

['Storefront',
 'Visible Sign of Business',
 'Business WiFi',
 'Business tables',
 'Business Plugs',
 'Business Internal CCTV',
 'External CCTV',
 'Number of female employees',
 'Accepts credit card',
 'Incline',
 'Width',
 'Ramp conditions',
 'Ramp // Crosswalk',
 'Ramp obstacles',
 'Crosswalk width',
 'Vertical signs',
 'Crossing length',
 'Access to people with disabilities',
 'Broken public property',
 'Incivility',
 'Obstacles',
 'Physical aspects in poor condition',
 'Poor infrastructure',
 'Positive elements',
 'Broken drain-cover',
 'Drug addicts',
 'Federal Judiciary',
 'Panic button',
 'Vandalism',
 'graffiti',
 'homelessness',
 'inclined ramp',
 'indication of security',
 'mobile vendors',
 'police presence',
 'risky building',
 'risky pavement',
 'risky road',
 'tourist map',
 'trash',
 'vehicle abandoned',
 'Coffee Shop',
 'Food joint',
 'Ice Cream Shop',
 'Lighting Stores',
 'Restaurant',
 'Open-Weekdays',
 'Open-Weekends',
 'Public WiFi',
 'number of bikes',
 'Mean Inhab

# Parameter Selection

## Weight


In [50]:
# Create dictionary of importance scores given by experts

# Get the column names as a list
column_names = df_normalized.columns.tolist()
column_names

importances= [10,7.5,5,6,9,7.75,8.5,10,10,8,4,6.5,8.5,9.5,9,8.5,6.5,10,9,9,8,9,9,9,10,10,5,
             3.75,10,4,8.25,7.5,5,5.5,2.5,10,10,9,9,10,3.5,10,10,7.5,1.5,10,9.5,10,8,5.5,
             4.5,7.5,10,3.5,3,10,10,8.5,10,6.5,10,2.5,2.5,6.5,4.5,9,10,8]

# Create the dictionary
importance_dict = {column_names[i]: importances[i] for i in range(len(column_names))}


print(importance_dict)

{'Storefront': 10, 'Visible Sign of Business': 7.5, 'Business WiFi': 5, 'Business tables': 6, 'Business Plugs': 9, 'Business Internal CCTV': 7.75, 'External CCTV': 8.5, 'Number of female employees': 10, 'Accepts credit card': 10, 'Incline': 8, 'Width': 4, 'Ramp conditions': 6.5, 'Ramp // Crosswalk': 8.5, 'Ramp obstacles': 9.5, 'Crosswalk width': 9, 'Vertical signs': 8.5, 'Crossing length': 6.5, 'Access to people with disabilities': 10, 'Broken public property': 9, 'Incivility': 9, 'Obstacles': 8, 'Physical aspects in poor condition': 9, 'Poor infrastructure': 9, 'Positive elements': 9, 'Broken drain-cover': 10, 'Drug addicts': 10, 'Federal Judiciary': 5, 'Panic button': 3.75, 'Vandalism': 10, 'graffiti': 4, 'homelessness': 8.25, 'inclined ramp': 7.5, 'indication of security': 5, 'mobile vendors': 5.5, 'police presence': 2.5, 'risky building': 10, 'risky pavement': 10, 'risky road': 9, 'tourist map': 9, 'trash': 10, 'vehicle abandoned': 3.5, 'Coffee Shop': 10, 'Food joint': 10, 'Ice Cre

In [51]:
importance_scores= np.array(importances)

We normalize weights so that they all equal 1  to ensure that the weights represent a distribution of importance across all the criteria.

In [52]:
normalized_weights = importance_scores / np.sum(importance_scores)

normalized_weights

array([0.01927711, 0.01445783, 0.00963855, 0.01156627, 0.0173494 ,
       0.01493976, 0.01638554, 0.01927711, 0.01927711, 0.01542169,
       0.00771084, 0.01253012, 0.01638554, 0.01831325, 0.0173494 ,
       0.01638554, 0.01253012, 0.01927711, 0.0173494 , 0.0173494 ,
       0.01542169, 0.0173494 , 0.0173494 , 0.0173494 , 0.01927711,
       0.01927711, 0.00963855, 0.00722892, 0.01927711, 0.00771084,
       0.01590361, 0.01445783, 0.00963855, 0.01060241, 0.00481928,
       0.01927711, 0.01927711, 0.0173494 , 0.0173494 , 0.01927711,
       0.00674699, 0.01927711, 0.01927711, 0.01445783, 0.00289157,
       0.01927711, 0.01831325, 0.01927711, 0.01542169, 0.01060241,
       0.0086747 , 0.01445783, 0.01927711, 0.00674699, 0.00578313,
       0.01927711, 0.01927711, 0.01638554, 0.01927711, 0.01253012,
       0.01927711, 0.00481928, 0.00481928, 0.01253012, 0.0086747 ,
       0.0173494 , 0.01927711, 0.01542169])

In [53]:
W= list(normalized_weights)

## Indifference Threshold

In [54]:
for column in df_normalized.columns:
    unique_values = df_normalized[column].unique()
    # Check if the unique values are only 0 and 1
    if set(unique_values) != {0, 1}:
        print(f"Unique values in column '{column}':")
        print(np.sort(unique_values))
        print()



Unique values in column 'Business WiFi':
[0.        0.8       0.9047619 1.       ]

Unique values in column 'External CCTV':
[0.  0.8 1. ]

Unique values in column 'Number of female employees':
[0.         0.05       0.1        0.12       0.13333333 0.2
 0.225      0.24       0.25       0.3        0.35       0.36
 0.4        0.41904762 0.45       0.5        0.7        1.        ]

Unique values in column 'Accepts credit card':
[0.   0.5  0.75]

Unique values in column 'Incline':
[0.         0.5        0.6        0.64285714 0.66666667 0.75
 1.        ]

Unique values in column 'Width':
[0.         0.5        0.75       0.83333333 0.92857143 0.95
 1.        ]

Unique values in column 'Ramp conditions':
[0.         0.5        0.66666667 0.83333333 0.85       0.92857143
 1.        ]

Unique values in column 'Ramp // Crosswalk':
[0.         0.5        0.57142857 0.66666667 0.8        1.        ]

Unique values in column 'Ramp obstacles':
[0.         0.1        0.14285714 0.33333333 0.5     

'Number of female employees' - 0.2

'Incline'  0.07

'Width' 0.1

'Ramp conditions' 0.1

'Ramp // Crosswalk' 0.1

Ramp obstacles' 0.05

'Restaurant' 0.2

'Open-Weekends' 0.1


In [70]:
# Create a dictionary where each key is a column name and each value is 0
indifference_dict = {column: 0 for column in column_names}

# Manually adjust the weights for the specific columns
indifference_dict['Number of female employees'] = 0.2
indifference_dict['Incline'] = 0.07
indifference_dict['Width'] = 0.1
indifference_dict['Ramp conditions'] = 0.1
indifference_dict['Ramp // Crosswalk'] = 0.1
indifference_dict['Ramp obstacles'] = 0.05
indifference_dict['Restaurant'] = 0.2
indifference_dict['Open-Weekends'] = 0.1

print(indifference_dict)


{'Storefront': 0, 'Visible Sign of Business': 0, 'Business WiFi': 0, 'Business tables': 0, 'Business Plugs': 0, 'Business Internal CCTV': 0, 'External CCTV': 0, 'Number of female employees': 0.2, 'Accepts credit card': 0, 'Incline': 0.07, 'Width': 0.1, 'Ramp conditions': 0.1, 'Ramp // Crosswalk': 0.1, 'Ramp obstacles': 0.05, 'Crosswalk width': 0, 'Vertical signs': 0, 'Crossing length': 0, 'Access to people with disabilities': 0, 'Broken public property': 0, 'Incivility': 0, 'Obstacles': 0, 'Physical aspects in poor condition': 0, 'Poor infrastructure': 0, 'Positive elements': 0, 'Broken drain-cover': 0, 'Drug addicts': 0, 'Federal Judiciary': 0, 'Panic button': 0, 'Vandalism': 0, 'graffiti': 0, 'homelessness': 0, 'inclined ramp': 0, 'indication of security': 0, 'mobile vendors': 0, 'police presence': 0, 'risky building': 0, 'risky pavement': 0, 'risky road': 0, 'tourist map': 0, 'trash': 0, 'vehicle abandoned': 0, 'Coffee Shop': 0, 'Food joint': 0, 'Ice Cream Shop': 0, 'Lighting Stores

In [74]:
Q = list(indifference_dict.values())

 ## Preference Threshold

In [55]:
np.sort(df_normalized['Crime Degree'].unique())

array([0.        , 0.2       , 0.33333333, 0.4       , 0.6       ,
       0.8       , 1.        ])

0.05

In [56]:
np.sort(df_normalized['Total coverage of transport'].unique())

array([0.        , 0.5       , 0.58333333, 0.66666667, 0.72222222,
       0.75      , 0.83333333, 0.91666667, 1.        ])

0.2

In [58]:
np.sort(df_normalized['lighting'].unique())

array([0.        , 0.33333333, 0.66666667, 1.        ])

0.3

In [59]:
np.sort(df_normalized['Ramp conditions'].unique())

array([0.        , 0.5       , 0.66666667, 0.83333333, 0.85      ,
       0.92857143, 1.        ])

0.3

In [60]:
np.sort(df_normalized['Ramp // Crosswalk'].unique())

array([0.        , 0.5       , 0.57142857, 0.66666667, 0.8       ,
       1.        ])

0.3

In [61]:
np.sort(df_normalized['Ramp obstacles'].unique())

array([0.        , 0.1       , 0.14285714, 0.33333333, 0.5       ,
       1.        ])

0.3

In [62]:
np.sort(df_normalized['Crosswalk width'].unique())

array([0.        , 0.33333333, 0.5       , 0.57142857, 0.66666667,
       0.8       , 1.        ])

0.3

In [75]:
# Create a dictionary where each key is a column name and each value is 0
preference_dict = {column: 0 for column in column_names}

# Manually adjust the weights for the specific columns
preference_dict['Total coverage of transport'] = 0.2
preference_dict['Crime Degree'] = 0.05
preference_dict['lighting'] = 0.3
preference_dict['Ramp conditions'] = 0.3
preference_dict['Ramp // Crosswalk'] = 0.3
preference_dict['Ramp obstacles'] = 0.3
preference_dict['Crosswalk width'] = 0.3


print(preference_dict)

{'Storefront': 0, 'Visible Sign of Business': 0, 'Business WiFi': 0, 'Business tables': 0, 'Business Plugs': 0, 'Business Internal CCTV': 0, 'External CCTV': 0, 'Number of female employees': 0, 'Accepts credit card': 0, 'Incline': 0, 'Width': 0, 'Ramp conditions': 0.3, 'Ramp // Crosswalk': 0.3, 'Ramp obstacles': 0.3, 'Crosswalk width': 0.3, 'Vertical signs': 0, 'Crossing length': 0, 'Access to people with disabilities': 0, 'Broken public property': 0, 'Incivility': 0, 'Obstacles': 0, 'Physical aspects in poor condition': 0, 'Poor infrastructure': 0, 'Positive elements': 0, 'Broken drain-cover': 0, 'Drug addicts': 0, 'Federal Judiciary': 0, 'Panic button': 0, 'Vandalism': 0, 'graffiti': 0, 'homelessness': 0, 'inclined ramp': 0, 'indication of security': 0, 'mobile vendors': 0, 'police presence': 0, 'risky building': 0, 'risky pavement': 0, 'risky road': 0, 'tourist map': 0, 'trash': 0, 'vehicle abandoned': 0, 'Coffee Shop': 0, 'Food joint': 0, 'Ice Cream Shop': 0, 'Lighting Stores': 0, 

In [76]:
P = list(preference_dict.values())

## Veto Threshold

In [63]:
np.sort(df_normalized['Crime Degree'].unique())

array([0.        , 0.2       , 0.33333333, 0.4       , 0.6       ,
       0.8       , 1.        ])

0.1

In [64]:
np.sort(df_normalized['Run over'].unique())

array([0.        , 0.33333333, 0.5       , 0.66666667, 1.        ])

0.1

In [65]:
np.sort(df_normalized['Deaths'].unique())

array([0.        , 0.5       , 0.66666667, 1.        ])

0.1

In [66]:
np.sort(df_normalized['Total coverage of transport'].unique())

array([0.        , 0.5       , 0.58333333, 0.66666667, 0.72222222,
       0.75      , 0.83333333, 0.91666667, 1.        ])

0.2

In [77]:
np.sort(df_normalized['Injuries'].unique())

array([0.        , 0.6       , 0.8       , 0.86666667, 1.        ])

0.1

In [78]:
# Create a dictionary where each key is a column name and each value is 0
veto_dict = {column: 0 for column in column_names}

# Manually adjust the weights for the specific columns
veto_dict['Run over'] = 0.1
veto_dict['Crime Degree'] = 0.1
veto_dict['Deaths'] = 0.1
veto_dict['Total coverage of transport'] = 0.2
veto_dict['Injuries'] = 0.1



print(veto_dict)

{'Storefront': 0, 'Visible Sign of Business': 0, 'Business WiFi': 0, 'Business tables': 0, 'Business Plugs': 0, 'Business Internal CCTV': 0, 'External CCTV': 0, 'Number of female employees': 0, 'Accepts credit card': 0, 'Incline': 0, 'Width': 0, 'Ramp conditions': 0, 'Ramp // Crosswalk': 0, 'Ramp obstacles': 0, 'Crosswalk width': 0, 'Vertical signs': 0, 'Crossing length': 0, 'Access to people with disabilities': 0, 'Broken public property': 0, 'Incivility': 0, 'Obstacles': 0, 'Physical aspects in poor condition': 0, 'Poor infrastructure': 0, 'Positive elements': 0, 'Broken drain-cover': 0, 'Drug addicts': 0, 'Federal Judiciary': 0, 'Panic button': 0, 'Vandalism': 0, 'graffiti': 0, 'homelessness': 0, 'inclined ramp': 0, 'indication of security': 0, 'mobile vendors': 0, 'police presence': 0, 'risky building': 0, 'risky pavement': 0, 'risky road': 0, 'tourist map': 0, 'trash': 0, 'vehicle abandoned': 0, 'Coffee Shop': 0, 'Food joint': 0, 'Ice Cream Shop': 0, 'Lighting Stores': 0, 'Restaur

In [79]:
V = list(veto_dict.values())